# Portability of AI libraries 
 


In [ ]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 62.0 MB/s 
     |████████████████████████████████| 212 kB 50.0 MB/s 
     |████████████████████████████████| 141 kB 71.2 MB/s 
     |████████████████████████████████| 101 kB 9.3 MB/s 
     |████████████████████████████████| 596 kB 57.1 MB/s 
     |████████████████████████████████| 127 kB 57.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from datasets import load_dataset

ds = load_dataset("codeparrot/github-code", streaming=True, split="train")
print(next(iter(ds)))

#EXPECTED OUTPUT:
{
 'code': "import mod189 from './mod189';\nvar value=mod189+1;\nexport default value;\n",
 'repo_name': 'MirekSz/webpack-es6-ts',
 'path': 'app/mods/mod190.js',
 'language': 'JavaScript',
 'license': 'isc',
 'size': 73
}


{'code': '\'use strict\';\n\nvar clear          = require(\'es5-ext/array/#/clear\')\n  , eIndexOf       = require(\'es5-ext/array/#/e-index-of\')\n  , setPrototypeOf = require(\'es5-ext/object/set-prototype-of\')\n  , callable       = require(\'es5-ext/object/valid-callable\')\n  , d              = require(\'d\')\n  , ee             = require(\'event-emitter\')\n  , Symbol         = require(\'es6-symbol\')\n  , iterator       = require(\'es6-iterator/valid-iterable\')\n  , forOf          = require(\'es6-iterator/for-of\')\n  , Iterator       = require(\'./lib/iterator\')\n  , isNative       = require(\'./is-native-implemented\')\n\n  , call = Function.prototype.call, defineProperty = Object.defineProperty\n  , SetPoly, getValues;\n\nmodule.exports = SetPoly = function (/*iterable*/) {\n\tvar iterable = arguments[0];\n\tif (!(this instanceof SetPoly)) return new SetPoly(iterable);\n\tif (this.__setData__ !== undefined) {\n\t\tthrow new TypeError(this + " cannot be reinitialized");\n\t}

{'code': "import mod189 from './mod189';\nvar value=mod189+1;\nexport default value;\n",
 'language': 'JavaScript',
 'license': 'isc',
 'path': 'app/mods/mod190.js',
 'repo_name': 'MirekSz/webpack-es6-ts',
 'size': 73}

In [ ]:
from datasets import load_dataset
from parso import parse

ds = load_dataset("codeparrot/github-code", streaming=True,
                  split="train", languages=["Python"])

iterator = iter(ds)
count = 0


def all_imports(node):
    for subscope in node.iter_funcdefs():
        yield from all_imports(subscope)
    for subscope in node.iter_classdefs():
        yield from all_imports(subscope)
    yield from node.iter_imports()


def contains_framework(ast):
    generator = all_imports(ast)
    for im in generator:
        # gets the first path from all imports
        name = im.get_paths()[0][0]
        if "torch" in name.__repr__() or "tensorflow" in name.__repr__() or "jax" in name.__repr__():
            return True
    return False


def main():
    files = []
    for i in range(100):
        code_data = next(iterator)
        if "torch" in code_data['code'] or "tensorflow" in code_data['code'] or "jax" in code_data['code']:
            files.append(code_data['code'])

    print("TEXT PASS LENGTH", len(files))
    filtered_files = [
        file for file in files if contains_framework(parse(file))
    ]

    print("AST PASS LENGTH", len(filtered_files))


main()

TEXT PASS LENGTH 2
AST PASS LENGTH 0


In [ ]:
print(ds.info.__dict__)

{'description': 'The GitHub Code dataest consists of 115M code files from GitHub in 32 programming languages with 60 extensions totalling in 1TB of text data. The dataset was created from the GitHub dataset on BiqQuery.\n', 'citation': '', 'homepage': 'https://cloud.google.com/blog/topics/public-datasets/github-on-bigquery-analyze-all-the-open-source-code/', 'license': "Multiple: see the 'license' field of each sample.", 'features': {'code': Value(dtype='string', id=None), 'repo_name': Value(dtype='string', id=None), 'path': Value(dtype='string', id=None), 'language': Value(dtype='string', id=None), 'license': Value(dtype='string', id=None), 'size': Value(dtype='int32', id=None)}, 'post_processed': None, 'supervised_keys': None, 'task_templates': None, 'builder_name': 'github-code', 'config_name': 'Python-all', 'version': 1.0.0, 'splits': None, 'download_checksums': None, 'download_size': None, 'post_processing_size': None, 'dataset_size': None, 'size_in_bytes': None}


In [ ]:
print(next(iter(ds)))

{'code': 'from django import forms\nfrom django.core.exceptions import ValidationError\nfrom django.core.validators import validate_slug\nfrom django.db import models\nfrom django.utils import simplejson as json\nfrom django.utils.text import capfirst\nfrom django.utils.translation import ugettext_lazy as _\n\nfrom philo.forms.fields import JSONFormField\nfrom philo.utils.registry import RegistryIterator\nfrom philo.validators import TemplateValidator, json_validator\n#from philo.models.fields.entities import *\n\n\nclass TemplateField(models.TextField):\n\t"""A :class:`TextField` which is validated with a :class:`.TemplateValidator`. ``allow``, ``disallow``, and ``secure`` will be passed into the validator\'s construction."""\n\tdef __init__(self, allow=None, disallow=None, secure=True, *args, **kwargs):\n\t\tsuper(TemplateField, self).__init__(*args, **kwargs)\n\t\tself.validators.append(TemplateValidator(allow, disallow, secure))\n\n\nclass JSONDescriptor(object):\n\tdef __init__(se